In [2]:
bind "set show-mode-in-prompt off"  # Turn off showing the vi mode in prompt, which clutters up the output here

# Project Management and Publishing with PDM

Notes from talk given to the WSU Python Working Group on February 8, 2023.

Here I show how to create a basic project with [PDM](https://pdm.fming.dev/latest/),
add dependencies and development dependencies, and publish the package on (Test)PyPI.

:::{note}
A companion repository with the example project created in these notes is available [here](https://github.com/edsq/eeskew-pwg-test-000).
:::

Requirements:

- The ability to get python executables of different versions, such as with [pyenv](https://github.com/pyenv/pyenv) or [conda](https://docs.conda.io/en/latest/miniconda.html)
- [PDM](https://pdm.fming.dev/latest/) available globally
- The [pdm-bump](https://github.com/carstencodes/pdm-bump) plugin installed

First, create the project directory and `cd` into it:

```bash
mkdir eeskew-pwg-test-000
cd eeskew-pwg-test-000
```

:::{important}
Because this is a throwaway test project, it is important that you give your project a name that won't conflict with any other package on PyPI or TestPyPI. Adding your name and some numbers is a good way to ensure this.
:::

## Set the python version and initialize the project

Here, we'll use python version 3.11, but you may change this to be whatever you like.

### Using `pyenv` (recommended)

Install python 3.11 if it is not already (see installed versions with `pyenv versions`):

```bash
pyenv install 3.11
```

Set the local python version for this project and initialize using that version:

```bash
pyenv local 3.11
pdm init --python python
```

`pyenv local` creates a file `.python-version` which `pyenv` uses to redirect the command `python` to the `python3.11`.  Thus, we only need to tell pdm to use the usual `python` executable.

### Using `conda`

Here, we'll use `conda` to get a particular python version, but we won't activate the conda environment (except to get a path to the `python` executable).  Environment management will be handled by PDM.

Get python 3.11:

```bash
conda create -y -p .conda_env python=3.11
pdm init --python .conda_env/bin/python
```

### `pdm init` options

For

```
Is the project a library that is installable?
A few more questions will be asked to include a project name and build backend [y/n] (n):
```

select `y`.  Otherwise, all the default options should be good.

In [3]:
# This cell hidden in presentation and docs
cd ../repos/eeskew-pwg-test-000
git clean -dfx  # remove all untracked files (src, build, dist, .venv)
git checkout $(git rev-list --topo-order main | tail -1)  # check out first commit
pdm venv create --force python

Removing .venv/
Removing build/
Removing dist/
Removing src/eeskew_pwg_test_000/__pycache__/
Note: checking out '1efdf5a51d8e08a89cf4eca801982f230f57a233'.

You are in 'detached HEAD' state. You can look around, make experimental
changes and commit them, and you can discard any commits you make in this
state without impacting any branches by performing another checkout.

If you want to create a new branch to retain commits you create, you may
do so (now or later) by using -b with the checkout command again. Example:

  git checkout -b <new-branch-name>

HEAD is now at 1efdf5a pdm init skeleton
⠴ Creating virtualenv using virtualenv.....
Virtualenv /Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000/.venv is created
successfully


In [4]:
# This cell hidden in presentation and docs
# Check that the environment and project are correct
pdm info

PDM version:
  2.4.6
Python Interpreter:
  /Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000/.venv/bin/python (3.11)
Project Root:
  /Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000
Project Packages:
  None


## The PDM project

Let's take a look at what we've created:

In [5]:
ls -a

.		.git		.pdm.toml	.venv		pyproject.toml
..		.gitignore	.python-version	README.md


The relevant files created are the `README.md`; `.pdm.toml`, which holds local configuration for this PDM project; and `pyproject.toml`, which holds project tool configuration and package metadata.

### The `pyproject.toml` file

In [6]:
cat pyproject.toml

[tool.pdm]

[project]
name = "eeskew-pwg-test-000"
version = "0.1.0"
description = "A test project for presentation to the WSU Python Working Group."
authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
]
dependencies = []
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}

[build-system]
requires = ["pdm-pep517>=1.0"]
build-backend = "pdm.pep517.api"


This file is written in `.toml` format, which stands for [Tom's Obvious Minimal Language](https://toml.io/en/).

The `tool.pdm` table is empty, although we'll add things here later on.

The `project` table contains the metadata needed to install our project.  Its values thus far were set by the options we chose while running `pdm init`.

The `build-system` section tells the build frontend (e.g. `pip`) what build backend to use - the build backend is what will actually create the distribution artifacts (wheels and sdists), which we'll see later.  See [PEP 517](https://peps.python.org/pep-0517/) for more information.  Here, we're just using the default PDM backend.

## Adding code

First we create our package directory in [src layout](https://hynek.me/articles/testing-packaging/), with an empty (for now) `__init__.py` file to indicate that it is a python package:

In [7]:
mkdir src
mkdir src/eeskew_pwg_test_000
touch src/eeskew_pwg_test_000/__init__.py

In [8]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was 1efdf5a pdm init skeleton
HEAD is now at a10844f Add __init__.py


### Add a module

Let's add some code in `src/eeskew_pwg_test_000/utils.py`:

In [9]:
# This cell hidden in presentation and docs
cat << "EOF" > src/eeskew_pwg_test_000/utils.py
def sarcasm(s):
    """Convert string `s` to sArCaSm TeXt."""
    out = ""
    for i, c in enumerate(s):
        if i % 2 == 0:
            out += c.lower()

        else:
            out += c.upper()

    return out
EOF

In [10]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was a10844f Add __init__.py
HEAD is now at 229305b Add utils.py


In [11]:
cat src/eeskew_pwg_test_000/utils.py

def sarcasm(s):
    """Convert string `s` to sArCaSm TeXt."""
    out = ""
    for i, c in enumerate(s):
        if i % 2 == 0:
            out += c.lower()

        else:
            out += c.upper()

    return out


The actual content of this code is not too important for the purposes of these notes, but for completeness, all it does is capitalize and lowercase alternating letters in a string.

## Install the project

To make our code available in the virtual environment, we have to install it:

In [12]:
pdm install

Lock file does not exist
Updating the lock file...
⠋ Fetching hashes for resolved packages...
🔒 Lock successful
Changes are written to pdm.lock.
⠋ Fetching hashes for resolved packages...
All packages are synced to date, nothing to do.
Installing the project as an editable package...
  ✔ Install eeskew-pwg-test-000 0.1.0 successful
36m⠹ Installing eeskew-pwg-test-000 0.1.0...
🎉 All complete!
36m⠹ Installing eeskew-pwg-test-000 0.1.0...



In [13]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was 229305b Add utils.py
HEAD is now at e12db04 Add pdm.lock after first pdm install


Now we can import our package:

In [14]:
pdm run python -c 'from eeskew_pwg_test_000.utils import sarcasm; print(sarcasm("Hello world!"))'

hElLo wOrLd!


Note we have to type `pdm run` before our command for it to be run within our project environment.

### The `pdm.lock` file

Running `pdm install` also created a new file, `pdm.lock`:

In [15]:
ls

README.md	pdm.lock	pyproject.toml	src


In [16]:
cat pdm.lock

# This file is @generated by PDM.
# It is not intended for manual editing.

[metadata]
lock_version = "4.1"
content_hash = "sha256:86165c41f17b4b263a688544a3ebc55eccc1713dd177c40649b2e936dab66751"

[metadata.files]


This is a lockfile, which will contain the exact versions of each project dependency we install.  It is useful for creating a perfect reproduction of the project virtual environment, which keeps our development reproducible over time and across different machines.

Right now, we have not installed anything other than the project itself, so it is essentially empty.

## Add a dependency

Let's add a dependency to our project:

In [17]:
pdm add cowsay

Adding packages to default dependencies: cowsay
⠹ Fetching hashes for resolved packages...
🔒 Lock successful
Changes are written to pdm.lock.
Changes are written to pyproject.toml.
⠋ Fetching hashes for resolved packages...
Synchronizing working set with lock file: 1 to add, 0 to update, 0 to remove

  ✔ Install cowsay 5.0 successful
Installing the project as an editable package...0m...
  ✔ Update eeskew-pwg-test-000 0.1.0 -> 0.1.0 successful
36m⠼ Updating eeskew-pwg-test-000 0.1.0 -> 0.1.0...
🎉 All complete!
36m⠼ Updating eeskew-pwg-test-000 0.1.0 -> 0.1.0...



In [18]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was e12db04 Add pdm.lock after first pdm install
HEAD is now at e5e9b85 Add cowsay to dependencies


### What did `pdm add` do?

`cowsay` now appears as a dependency in `pyproject.toml`:

In [19]:
git diff HEAD~ pyproject.toml | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: pyproject.toml
────────────────────────────────────────────────────────────────────────────────
@ pyproject.toml:10 @ description = "A test project for presentation to the WSU Python Working Group."
authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
]
dependencies = []
dependencies = [
    "cowsay>=5.0",
]
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}


We've also updated `pdm.lock` to include cowsay:

In [20]:
cat pdm.lock

# This file is @generated by PDM.
# It is not intended for manual editing.

[[package]]
name = "cowsay"
version = "5.0"
summary = "The famous cowsay for GNU/Linux is now available for python"

[metadata]
lock_version = "4.1"
content_hash = "sha256:a04c8eb9409090bc9acb94e5cec5ef19afdecacb2c169628142af5ca472135f0"

[metadata.files]
"cowsay 5.0" = [
    {url = "https://files.pythonhosted.org/packages/6b/b8/9f497fd045d74fe21d91cbe8debae0b451229989e35b539d218547d79fc6/cowsay-5.0.tar.gz", hash = "sha256:c00e02444f5bc7332826686bd44d963caabbaba9a804a63153822edce62bbbf3"},
]


We can now import `cowsay`:

In [21]:
pdm run python -c 'import cowsay; cowsay.cow("moo!")'

  ____
| moo! |
  ====
    \
     \
       ^__^
       (oo)\_______
       (__)\       )\/\
           ||----w |
           ||     ||


### Adding more code

Let's add a new function to `utils.py`:

In [22]:
# This cell hidden in presentation and docs
cat << "EOF" > src/eeskew_pwg_test_000/utils.py
import cowsay

def sarcasm(s):
    """Convert string `s` to sArCaSm TeXt."""
    out = ""
    for i, c in enumerate(s):
        if i % 2 == 0:
            out += c.lower()

        else:
            out += c.upper()

    return out

def sarcastic_cowsay(s):
    """Cowsay `s`, sArCaStIcAlLy."""
    sarcastic_s = sarcasm(s)
    cowsay.cow(sarcastic_s)
EOF

In [23]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was e5e9b85 Add cowsay to dependencies
HEAD is now at fe97875 Add sarcastic_cowsay to utils


In [24]:
git diff HEAD~ src/eeskew_pwg_test_000/utils.py | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: src/eeskew_pwg_test_000/utils.py
────────────────────────────────────────────────────────────────────────────────
@ src/eeskew_pwg_test_000/utils.py:2 @
import cowsay
 
def sarcasm(s):
    """Convert string `s` to sArCaSm TeXt."""
    out = ""
@ src/eeskew_pwg_test_000/utils.py:14 @ def sarcasm(s):
            out += c.upper()

    return out
 
def sarcastic_cowsay(s):
    """Cowsay `s`, sArCaStIcAlLy."""
    sarcastic_s = sarcasm(s)
    cowsay.cow(sarcastic_s)


We can now run this new function:

In [25]:
pdm run python -c 'from eeskew_pwg_test_000.utils import sarcastic_cowsay; sarcastic_cowsay("mooo!")'

  _____
| mOoO! |
  =====
     \
      \
        ^__^
        (oo)\_______
        (__)\       )\/\
            ||----w |
            ||     ||


:::{note}
We didn't have to re-run `pdm install` to use our new function - this is because PDM installs our `eeskew_pwg_test_000` package in "editable mode", which acts sort of like a symlink between the source code and the installed files in the `.venv` directory.
:::

## Add a development dependency

The dependencies listed in the `project.dependencies` section of `pyproject.toml` will all be installed when someone runs `pip install eeskew_pwg_test_project`.  What if we have dependencies we only want in our development environment?

Let's add `black`, a tool to automatically format our code:

In [26]:
pdm add -d black

Adding packages to dev dev-dependencies: black
⠙ Fetching hashes for resolved packages...m23.1.0
🔒 Lock successful
Changes are written to pdm.lock.
Changes are written to pyproject.toml.
⠋ Fetching hashes for resolved packages...
Synchronizing working set with lock file: 6 to add, 0 to update, 0 to remove

  ⠋ Installing black 23.1.0...
  ⠋ Installing black 23.1.0...         
  ⠋ Installing click 8.1.3...          
  ⠋ Installing mypy-extensions 1.0.0...
  ⠋ Installing black 23.1.0...         
  ⠋ Installing click 8.1.3...          
  ⠋ Installing mypy-extensions 1.0.0...
  ⠋ Installing black 23.1.0...         
  ⠋ Installing click 8.1.3...          
  ⠋ Installing mypy-extensions 1.0.0...
  ⠋ Installing packaging 23.0...       
  ⠋ Installing black 23.1.0...         
  ⠋ Installing click 8.1.3...          
  ⠋ Installing mypy-extensions 1.0.0...
  ⠋ Installing packaging 23.0...       
  ⠋ Installing pathspec 0.11.0...      
  ⠙ Installing black 23.1.0...         
  ⠙ Installing click 

In [27]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was fe97875 Add sarcastic_cowsay to utils
HEAD is now at c545db9 Add black to dev dependencies


### What happened?

In [28]:
git diff HEAD~ pyproject.toml | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: pyproject.toml
────────────────────────────────────────────────────────────────────────────────
@ pyproject.toml:4 @
[tool.pdm]
[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",
]

[project]
name = "eeskew-pwg-test-000"


We've added a new `[dev-dependencies]` sub-table to the `[tool.pdm]` table.  When we run `pdm install`, by default, PDM will install dependencies from here in addition to the dependencies listed in `[project.dependencies]`.  A different tool like `pip`, however, will not.

See the [PDM docs](https://pdm.fming.dev/latest/usage/dependency/#add-development-only-dependencies) for more information on development dependencies.

The lockfile has also been updated:

In [29]:
cat pdm.lock

# This file is @generated by PDM.
# It is not intended for manual editing.

[[package]]
name = "black"
version = "23.1.0"
requires_python = ">=3.7"
summary = "The uncompromising code formatter."
dependencies = [
    "click>=8.0.0",
    "mypy-extensions>=0.4.3",
    "packaging>=22.0",
    "pathspec>=0.9.0",
    "platformdirs>=2",
]

[[package]]
name = "click"
version = "8.1.3"
requires_python = ">=3.7"
summary = "Composable command line interface toolkit"
dependencies = [
    "colorama; platform_system == \"Windows\"",
]

[[package]]
name = "colorama"
version = "0.4.6"
requires_python = "!=3.0.*,!=3.1.*,!=3.2.*,!=3.3.*,!=3.4.*,!=3.5.*,!=3.6.*,>=2.7"
summary = "Cross-platform colored terminal text."

[[package]]
name = "cowsay"
version = "5.0"
summary = "The famous cowsay for GNU/Linux is now available for python"

[[package]]
name = "mypy-extensions"
version = "1.0.0"
requires_python = ">=3.5"
summary = "Type system extensions for programs checked with the mypy type checker."

[[package

    {url = "https://files.pythonhosted.org/packages/c2/f1/df59e28c642d583f7dacffb1e0965d0e00b218e0186d7858ac5233dce840/click-8.1.3-py3-none-any.whl", hash = "sha256:bb4d8133cb15a609f44e8213d9b391b0809795062913b383c62be0ee95b1db48"},
]
"colorama 0.4.6" = [
    {url = "https://files.pythonhosted.org/packages/d1/d6/3965ed04c63042e047cb6a3e6ed1a63a35087b6a609aa3a15ed8ac56c221/colorama-0.4.6-py2.py3-none-any.whl", hash = "sha256:4f1d9991f5acc0ca119f9d443620b77f9d6b33703e51011c16baf57afb285fc6"},
    {url = "https://files.pythonhosted.org/packages/d8/53/6f443c9a4a8358a93a6792e2acffb9d9d5cb0a5cfd8802644b7b1c9a02e4/colorama-0.4.6.tar.gz", hash = "sha256:08695f5cb7ed6e0531a20572697297273c47b8cae5a63ffc6d6ed5c201be6e44"},
]
"cowsay 5.0" = [
    {url = "https://files.pythonhosted.org/packages/6b/b8/9f497fd045d74fe21d91cbe8debae0b451229989e35b539d218547d79fc6/cowsay-5.0.tar.gz", hash = "sha256:c00e02444f5bc7332826686bd44d963caabbaba9a804a63153822edce62bbbf3"},
]
"mypy-extensions 1.0.0" = [
    {ur

Many new packages now exist in the lockfile, not just `cowsay` and `black`.  This is because (unlike `cowsay`), `black` itself has dependencies that we needed to install to get it to work.  This lock file records exactly the versions of those sub-dependencies that we've now installed into our project virtual environment.

When we run `pdm install`, if the lock file exists (and `pyproject.toml` hasn't been changed since the lockfile was last updated), PDM will install precisely the packages listed in the lockfile, so we'll always be working in the same virtual environment.  This is useful for developing and testing the code, so you should always include the lockfile in your project version control.

However, we don't want to impose these restrictions on users of our library, or our project would rapidly become impossible to install due to other packages requiring different versions of the packages in the lockfile.  The only thing that we care about is that users have the right versions of the dependencies we directly use, which are listed in the `project.dependencies` array in `pyproject.toml`.  This is why `pip install` does not care about the existence of the lockfile.

### Using black

We can now run `black` within our environment.  Let's re-write our code with poor formatting (note the spacing around the `==`, `%`, and `+=` operators), and then run `black` on it:

In [30]:
# This cell hidden in presentation and docs
cat << "EOF" > src/eeskew_pwg_test_000/utils.py
import cowsay

def sarcasm(s):
    """Convert string `s` to sArCaSm TeXt."""
    out = ''
    for i,c in enumerate(s):
        if i% 2 ==0:
            out +=c.lower()

        else:
            out+= c.upper()

    return out

def sarcastic_cowsay(s):
    """Cowsay `s`, sArCaStIcAlLy."""
    sarcastic_s = sarcasm(s)
    cowsay.cow(sarcastic_s)
EOF

In [31]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was c545db9 Add black to dev dependencies
HEAD is now at b0ccee2 Use bad formatting in utils for black to fix


In [32]:
git diff HEAD~ src/eeskew_pwg_test_000/utils.py | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: src/eeskew_pwg_test_000/utils.py
────────────────────────────────────────────────────────────────────────────────
@ src/eeskew_pwg_test_000/utils.py:5 @ import cowsay

def sarcasm(s):
    """Convert string `s` to sArCaSm TeXt."""
    out = ""
    for i, c in enumerate(s):
        if i % 2 == 0:
            out += c.lower()
    out = ''
    for i,c in enumerate(s):
        if i% 2 ==0:
            out +=c.lower()

        else:
            out += c.upper()
            out+= c.upper()

    return out



In [33]:
pdm run black src/

reformatted /Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000/src/eeskew_pwg_test_000/utils.py

All done! ✨ 🍰 ✨
1 file reformatted, 1 file left unchanged.


In [34]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was b0ccee2 Use bad formatting in utils for black to fix
HEAD is now at be4114d Fix formatting with black


### What happened?

In [35]:
git diff HEAD~ src/eeskew_pwg_test_000/utils.py | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: src/eeskew_pwg_test_000/utils.py
────────────────────────────────────────────────────────────────────────────────
@ src/eeskew_pwg_test_000/utils.py:4 @
import cowsay

 
def sarcasm(s):
    """Convert string `s` to sArCaSm TeXt."""
    out = ''
    for i,c in enumerate(s):
        if i% 2 ==0:
            out +=c.lower()
    out = ""
    for i, c in enumerate(s):
        if i % 2 == 0:
            out += c.lower()

        else:
            out+= c.upper()
            out += c.upper()

    return out

 
def sarcastic_cowsay(s):
    """Cowsay `s`, sArCaStIcAlLy."""
    sarcastic_s = sarcasm(s)


`black` has automatically re-formatted our code, fixing the poor formatting we introduced earlier.

:::{seealso}
See the [black documentation](https://black.readthedocs.io/en/stable/) for more information.
:::

## Package version

Right now, the package version (`"0.1.0"`) is stored in the `pyproject.toml` file (in the `project.version` keyword).  The best practice is to place this in a `__version__.py` file, and have that be the single source of truth for our package version.

Create a `__version__.py` file in `src/eeskew_pwg_test_000`, and add the `__version__` variable to it.  The new file should look like:

In [36]:
# This cell hidden in presentation and docs
echo '__version__ = "0.1.0"' > src/eeskew_pwg_test_000/__version__.py

In [37]:
cat src/eeskew_pwg_test_000/__version__.py

__version__ = "0.1.0"


Now modify the `pyproject.toml` file so that the version is [dynamic metadata](https://packaging.python.org/en/latest/specifications/declaring-project-metadata/#dynamic):

In [38]:
# This cell hidden in presentation and docs
cat << EOF > pyproject.toml
[tool.pdm]
version = { source = "file", path = "src/eeskew_pwg_test_000/__version__.py" }

[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",
]

[project]
name = "eeskew-pwg-test-000"
description = "A test project for presentation to the WSU Python Working Group."
authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
]
dependencies = [
    "cowsay>=5.0",
]
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}
dynamic = ["version"]

[build-system]
requires = ["pdm-pep517>=1.0"]
build-backend = "pdm.pep517.api"
EOF

In [39]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was be4114d Fix formatting with black
HEAD is now at cc5aee4 Use dynamic versioning in pyproject.toml


In [40]:
git diff HEAD~ pyproject.toml | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: pyproject.toml
────────────────────────────────────────────────────────────────────────────────
@ pyproject.toml:3 @
[tool.pdm]
version = { source = "file", path = "src/eeskew_pwg_test_000/__version__.py" }
 
[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",
@ pyproject.toml:11 @ dev = [

[project]
name = "eeskew-pwg-test-000"
version = "0.1.0"
description = "A test project for presentation to the WSU Python Working Group."
authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
@ pyproject.toml:21 @ dependencies = [
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}
dynamic = ["version"]

[build-system]
requires = ["pdm-pep517>=1.0"]


Note the new `project.dynamic` array, the new `tool.pdm.version` table, and that the `project.version` key is gone.

We can check our current version like so:

In [41]:
pdm show --version

0.1.0


We should also add the `__version__` variable to our `__init__.py`:

In [42]:
# This cell hidden in presentation and docs
echo 'from eeskew_pwg_test_000.__version__ import __version__' > src/eeskew_pwg_test_000/__init__.py

In [43]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was cc5aee4 Use dynamic versioning in pyproject.toml
HEAD is now at 32aec80 Add __version__ to __init__


In [44]:
git diff HEAD~ src/eeskew_pwg_test_000/__init__.py | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: src/eeskew_pwg_test_000/__init__.py
────────────────────────────────────────────────────────────────────────────────
@ src/eeskew_pwg_test_000/__init__.py:1 @
from eeskew_pwg_test_000.__version__ import __version__


This allows us to check the version from python in the conventional way:

In [45]:
pdm run python -c "import eeskew_pwg_test_000; print(eeskew_pwg_test_000.__version__)"

0.1.0


## Packaging the project

Let's review the project as it exists so far:

In [46]:
tree

.
├── README.md
├── pdm.lock
├── pyproject.toml
└── src
    └── eeskew_pwg_test_000
        ├── __init__.py
        ├── __pycache__
        │   ├── __init__.cpython-311.pyc
        │   ├── __version__.cpython-311.pyc
        │   └── utils.cpython-311.pyc
        ├── __version__.py
        └── utils.py

3 directories, 9 files


To create a [sdist](https://packaging.python.org/en/latest/specifications/source-distribution-format/) and [wheel](https://packaging.python.org/en/latest/specifications/binary-distribution-format/):

In [47]:
pdm build

Building sdist...
Built sdist at 
/Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000/dist/eeskew-pwg-test-000-0.
1.0.tar.gz
Built wheel at 
/Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000/dist/eeskew_pwg_test_000-0.
1.0-py3-none-any.whl


### What happened?

We've created a new directory named `dist`, where these two distribution formats have been placed.

In [48]:
tree

.
├── README.md
├── build
├── dist
│   ├── eeskew-pwg-test-000-0.1.0.tar.gz
│   └── eeskew_pwg_test_000-0.1.0-py3-none-any.whl
├── pdm.lock
├── pyproject.toml
└── src
    └── eeskew_pwg_test_000
        ├── __init__.py
        ├── __pycache__
        │   ├── __init__.cpython-311.pyc
        │   ├── __version__.cpython-311.pyc
        │   └── utils.cpython-311.pyc
        ├── __version__.py
        └── utils.py

5 directories, 11 files


We could install this project into a different python environment with `python -m pip install dist/eeskew-pwg-test-000-0.1.0.tar.gz` or `python -m pip install dist/eeskew_pwg_test_000-0.1.0-py3-none-any.whl` (the latter is faster).

## Publishing the project

Now we'll publish the project on (Test)PyPI.

:::{note}
To publish on the actual index (PyPI, not TestPyPI), simply replace `testpypi` with `pypi` in the instructions that follow.  Try not to pollute PyPI with throwaway projects!
:::

### Setting up PyPI credentials

0. First, make an account on [TestPyPI](https://test.pypi.org).
1. Navigate to your account settings, scroll down to "API tokens", and click "Add API token"
2. Give the token a descriptive name, set the scope to "Entire account (all projects)", and click "Add token".
3. Copy the token that appears - heeding the warning that it will appear only once!
4. Now we'll configure PDM with these credentials (replacing `<PASTE_YOUR_TOKEN_HERE>` with the token you've just copied):

```bash
pdm config repository.testpypi.username "__token__"
pdm config repository.testpypi.password "<PASTE_YOUR_TOKEN_HERE>"
```

### Setting up a test-publish PDM script

To publish on PyPI, we could now simply run:

```bash
pdm publish -r testpypi
```

Note that you do not need to run `pdm build` first - PDM will build the distribution as part of `publish` anyway.

However, TestPyPI won't let you overwrite an existing version of your package, so we have to bump our version every time we want to do this.  Let's set up a PDM script to automate that.

:::{attention}
If you are publishing on PyPI (*not* TestPyPI), you probably don't want to use this script.  Publishing will be as simple as running `pdm bump {version}` to increment your package version number (for example, `pdm bump patch`), and then publishing with `pdm publish` (equivalent to `pdm publish -r pypyi`).
:::

First, ensure you have the [pdm-bump](https://github.com/carstencodes/pdm-bump) plugin installed.

We add a new PDM script in the `tool.pdm.scripts` table of `pyproject.toml`:

In [49]:
# This cell hidden in presentation and docs
cat << "EOF" > pyproject.toml
[tool.pdm]
version = { source = "file", path = "src/eeskew_pwg_test_000/__version__.py" }

[tool.pdm.scripts]
test-publish.shell = '''\
VERSION=$(pdm show --version)
pdm bump patch > /dev/null
DEV_VERSION=$(pdm show --version).dev$(date +%s)
echo "__version__ = \"$DEV_VERSION\"" > src/eeskew_pwg_test_000/__version__.py
pdm publish -r testpypi
echo "__version__ = \"$VERSION\"" > src/eeskew_pwg_test_000/__version__.py
'''

[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",
]

[project]
name = "eeskew-pwg-test-000"
description = "A test project for presentation to the WSU Python Working Group."
authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
]
dependencies = [
    "cowsay>=5.0",
]
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}
dynamic = ["version"]

[build-system]
requires = ["pdm-pep517>=1.0"]
build-backend = "pdm.pep517.api"
EOF

In [50]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was 32aec80 Add __version__ to __init__
HEAD is now at a164016 Add test-publish PDM script


In [51]:
git diff HEAD~ pyproject.toml | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: pyproject.toml
────────────────────────────────────────────────────────────────────────────────
@ pyproject.toml:4 @
[tool.pdm]
version = { source = "file", path = "src/eeskew_pwg_test_000/__version__.py" }

[tool.pdm.scripts]
test-publish.shell = '''\
VERSION=$(pdm show --version)
pdm bump patch > /dev/null
DEV_VERSION=$(pdm show --version).dev$(date +%s)
echo "__version__ = \"$DEV_VERSION\"" > src/eeskew_pwg_test_000/__version__.py
pdm publish -r testpypi
echo "__version__ = \"$VERSION\"" > src/eeskew_pwg_test_000/__version__.py
'''
 
[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",


When we run `pdm run test-publish`, this script:
1. Gets the current version with `pdm show --version`
2. Changes the package version to a patch bump of that version with `.dev{date in seconds}` appended.  This is a [developmental release](https://peps.python.org/pep-0440/#developmental-releases) format.
3. Publishes the package on TestPyPI
4. Returns the package version to its original value

Let's run it!

In [52]:
pdm run test-publish

Building sdist...
Built sdist at 
/Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000/dist/eeskew-pwg-test-000-0.
1.1.dev1677485299.tar.gz
Built wheel at 
/Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000/dist/eeskew_pwg_test_000-0.
1.1.dev1677485299-py3-none-any.whl
Uploading eeskew_pwg_test_000-0.1.1.dev1677485299-py3-none-any.whl
Uploading eeskew-pwg-test-000-0.1.1.dev1677485299.tar.gz━━━━━━━━ 4.7/4.7 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 kB • 00:00 • ?
 100% ━━━━━━━

## Other project metadata

There is quite a bit of project metadata we can specify in `pyproject.toml` - here are some other examples.

:::{seealso}
See the [PyPA specification](https://packaging.python.org/en/latest/specifications/declaring-project-metadata/) for a complete list of possible metadata keys.
:::

### Add an entry point to the package

If we're developing a command-line application, we want our users to be able to run the application with a single command, not something like `python path_to_script/script.py`.  We can enable this by adding an [entry point](https://packaging.python.org/en/latest/specifications/declaring-project-metadata/#entry-points) to `pyproject.toml`.

First, add a new module and new function:

In [53]:
# This cell hidden in presentation and docs
cat << "EOF" > src/eeskew_pwg_test_000/cli.py
import argparse

from eeskew_pwg_test_000.utils import sarcastic_cowsay

def main():
    """Cowsay something sarcastically from the command line."""
    parser = argparse.ArgumentParser()
    parser.add_argument("speech")
    args = parser.parse_args()
    
    s = args.speech
    sarcastic_cowsay(s)
EOF

In [54]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was a164016 Add test-publish PDM script
HEAD is now at 16a7c4f Add cli.py


In [55]:
cat src/eeskew_pwg_test_000/cli.py

import argparse

from eeskew_pwg_test_000.utils import sarcastic_cowsay

def main():
    """Cowsay something sarcastically from the command line."""
    parser = argparse.ArgumentParser()
    parser.add_argument("speech")
    args = parser.parse_args()
    
    s = args.speech
    sarcastic_cowsay(s)


Now let's add the script to `pyproject.toml`:

In [56]:
# This cell hidden in presentation and docs
cat << "EOF" > pyproject.toml
[tool.pdm]
version = { source = "file", path = "src/eeskew_pwg_test_000/__version__.py" }

[tool.pdm.scripts]
test-publish.shell = '''\
VERSION=$(pdm show --version)
pdm bump patch > /dev/null
DEV_VERSION=$(pdm show --version).dev$(date +%s)
echo "__version__ = \"$DEV_VERSION\"" > src/eeskew_pwg_test_000/__version__.py
pdm publish -r testpypi
echo "__version__ = \"$VERSION\"" > src/eeskew_pwg_test_000/__version__.py
'''

[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",
]

[project]
name = "eeskew-pwg-test-000"
description = "A test project for presentation to the WSU Python Working Group."
authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
]
dependencies = [
    "cowsay>=5.0",
]
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}
dynamic = ["version"]

[project.scripts]
sarcasticow = "eeskew_pwg_test_000.cli:main"

[build-system]
requires = ["pdm-pep517>=1.0"]
build-backend = "pdm.pep517.api"
EOF

In [57]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was 16a7c4f Add cli.py
HEAD is now at 7bd5744 Add sarcasticow entry point to pyproject.toml


In [58]:
git diff HEAD~ pyproject.toml | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: pyproject.toml
────────────────────────────────────────────────────────────────────────────────
@ pyproject.toml:33 @ readme = "README.md"
license = {text = "MIT"}
dynamic = ["version"]

[project.scripts]
sarcasticow = "eeskew_pwg_test_000.cli:main"
 
[build-system]
requires = ["pdm-pep517>=1.0"]
build-backend = "pdm.pep517.api"


In [59]:
pdm install

⠋ Fetching hashes for resolved packages...
All packages are synced to date, nothing to do.
Installing the project as an editable package...
  ✔ Update eeskew-pwg-test-000 0.1.0 -> 0.1.0 successful
36m⠦ Updating eeskew-pwg-test-000 0.1.0 -> 0.1.0...
🎉 All complete!
36m⠦ Updating eeskew-pwg-test-000 0.1.0 -> 0.1.0...



Now we can run our command from within the environment:

In [60]:
pdm run sarcasticow "I'm a sarcastic cow"

  ___________________
| i'm a sArCaStIc cOw |
                   \
                    \
                      ^__^
                      (oo)\_______
                      (__)\       )\/\
                          ||----w |
                          ||     ||


:::{note}
Because we're using `pdm` for environment management, we still need to use `pdm run` to access the script installed into the virtual environment.

Unlike the `test-publish` PDM script we wrote earlier, if we activated the environment with `source .venv/bin/activate`, we could simply use the `sarcasticow` command by itself, and users who install our package with `pip` into their own virtualenv or conda environment will also have access to `sarcasticow`.  Even better, users who install our package through [`pipx`](https://pypa.github.io/pipx/) will be able to use the `sarcasticow` command without activating a virtual environment.
:::

### Updating README

Thus far, we've left our README as an empty file.  This is bad.  Let's update it to show our utility's usage:

In [61]:
# This cell hidden in presentation and docs
cat << "EOF" > README.md
# eeskew-pwg-test-000

This is a companion repository to the presentation [Project Management and Publishing with PDM](https://edsq.github.io/eds-notes/pwg_presentation_02-08-2023.html).

Command-line usage:

```
$ sarcasticow "I'm a sarcastic cow"

  ___________________
| i'm a sArCaStIc cOw |
  ===================
                   \
                    \
                      ^__^
                      (oo)\_______
                      (__)\       )\/\
                          ||----w |
                          ||     ||

```
EOF

In [62]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was 7bd5744 Add sarcasticow entry point to pyproject.toml
HEAD is now at 2a258c3 Add sarcasticow to README


In [63]:
git diff HEAD~ README.md | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: README.md
────────────────────────────────────────────────────────────────────────────────
@ README.md:4 @
# eeskew-pwg-test-000
 
This is a companion repository to the presentation [Project Management and Publishing with PDM](https://edsq.github.io/eds-notes/pwg_presentation_02-08-2023.html).
 
Command-line usage:
 
```
$ sarcasticow "I'm a sarcastic cow"
 
  ___________________
| i'm a sArCaStIc cOw |
                   \
                    \
                      ^__^
                      (oo)\_______
                      (__)\       )\/\
                          ||----w |
                          ||     ||
 
```


### Project URLs

We can add relevant URLs in the [`urls` table](https://packaging.python.org/en/latest/specifications/declaring-project-metadata/#urls), which will appear in the sidebar on PyPI.

In [66]:
# This cell hidden in presentation and docs
cat << "EOF" > pyproject.toml
[tool.pdm]
version = { source = "file", path = "src/eeskew_pwg_test_000/__version__.py" }

[tool.pdm.scripts]
test-publish.shell = '''\
VERSION=$(pdm show --version)
pdm bump patch > /dev/null
DEV_VERSION=$(pdm show --version).dev$(date +%s)
echo "__version__ = \"$DEV_VERSION\"" > src/eeskew_pwg_test_000/__version__.py
pdm publish -r testpypi
echo "__version__ = \"$VERSION\"" > src/eeskew_pwg_test_000/__version__.py
'''

[tool.pdm.dev-dependencies]
dev = [
    "black>=23.1.0",
]

[project]
name = "eeskew-pwg-test-000"
description = "A test project for presentation to the WSU Python Working Group."
authors = [
    {name = "Edward Eskew", email = "edward.eskew@wsu.edu"},
]
dependencies = [
    "cowsay>=5.0",
]
requires-python = ">=3.11"
readme = "README.md"
license = {text = "MIT"}
dynamic = ["version"]

[project.urls]
Documentation = "https://edsq.github.io/eds-notes/pwg_presentation_02-08-2023.html"
Repository = "https://github.com/edsq/eeskew-pwg-test-000"

[project.scripts]
sarcasticow = "eeskew_pwg_test_000.cli:main"

[build-system]
requires = ["pdm-pep517>=1.0"]
build-backend = "pdm.pep517.api"
EOF

In [68]:
# checkpoint
git add -A
git checkout $(git rev-list --topo-order HEAD...main | tail -1)  # check out next commit

Previous HEAD position was 2a258c3 Add sarcasticow to README
HEAD is now at 58a0a99 Add urls to project metadata


In [71]:
git diff HEAD~ pyproject.toml | ../../scripts/diff-so-fancy

────────────────────────────────────────────────────────────────────────────────
modified: pyproject.toml
────────────────────────────────────────────────────────────────────────────────
@ pyproject.toml:33 @ readme = "README.md"
license = {text = "MIT"}
dynamic = ["version"]

[project.urls]
Documentation = "https://edsq.github.io/eds-notes/pwg_presentation_02-08-2023.html"
Repository = "https://github.com/edsq/eeskew-pwg-test-000"
 
[project.scripts]
sarcasticow = "eeskew_pwg_test_000.cli:main"



Finally, we publish again to TestPYPI:

In [67]:
pdm run test-publish

Building sdist...
Built sdist at 
/Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000/dist/eeskew-pwg-test-000-0.
1.1.dev1677485469.tar.gz
Built wheel at 
/Users/Ed/python/eds-notes/repos/eeskew-pwg-test-000/dist/eeskew_pwg_test_000-0.
1.1.dev1677485469-py3-none-any.whl
Uploading eeskew_pwg_test_000-0.1.1.dev1677485469-py3-none-any.whl
Uploading eeskew-pwg-test-000-0.1.1.dev1677485469.tar.gz━━━━━━━━ 6.5/6.5 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 kB • 00:00 • ?
 100% ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.5/6.5 kB • 00:00 • ?
 100% ━━━━━━━

Our page on TestPyPI now shows the README, and the documentation and repository links are available on the sidebar.

## Conclusion

And that's it!  We've gone through the basics of project management, packaging, and publishing on (Test)PyPI.